In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import transformers
from PIL import Image
import os
import glob
import wandb
from torchinfo import summary
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns 
import SimpleITK as sitk
from utils import normalize_sitk_image, MRI_Dataset_within_ROI, MRI_Dataset_within_ROI_both_prepost
from torchvision.transforms import v2
import torchio as tio
from torchmetrics import Accuracy, Recall, Precision, F1Score
torch.set_num_threads(24)
DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

from sklearn.model_selection import train_test_split

In [74]:
# parameters
SIZE = (100,100,100)
TRAIN_SPLIT = 0.75
SPLIT_SEED = 123456
BATCH_SIZE = 25

DATA_PATH = '../../../Processed NIFTI Dataset/'
CLASSIFICATION = 'ER'
dataset_path = f'../../Train Test Splits/{CLASSIFICATION}/'

MODEL_SAVE_PATH = f'basic_model_{CLASSIFICATION}.pth'
PROJECT_NAME = 'Breast Cancer Subtype Prediction'

NUM_WORKERS = 20

In [75]:
# dataset loading

train_set = pd.read_csv(dataset_path + 'train.csv')
test_set  = pd.read_csv(dataset_path + 'test.csv')

bounding_boxes = pd.read_csv('../../Data/segmentation_annotations_NIFTI.csv').set_index('Patient_ID')

NUM_CLASSES = len(train_set['label'].unique())

In [76]:
train_set

,Unnamed: 0,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,label
0,92,0.962565,0.853207,12.531504,14.687523,16.001639,16.504796,17.428449,20.185653,1655.774227,...,7.903733e-08,-3.203427e-16,0.000281,0.000000e+00,0.0,1000000.0,0.0,0.0,0.0,1
1,777,0.795114,0.575234,41.275598,71.754477,81.554735,89.356894,106.275846,109.381063,80893.671683,...,4.901414e-01,2.902818e+00,0.001373,6.014770e+07,0.0,1000000.0,0.0,0.0,0.0,0
2,580,0.674057,0.509993,20.299251,39.802964,32.193327,46.356854,49.344909,50.722644,9592.051652,...,5.530019e-01,2.300814e+00,0.001829,8.841581e+06,0.0,1000000.0,0.0,0.0,0.0,1
3,858,0.460551,0.364788,19.775146,54.209994,30.968839,49.615171,49.696789,53.641526,9223.914005,...,6.344686e-01,2.249814e+00,0.002600,6.205718e+06,0.0,1000000.0,0.0,0.0,0.0,1
4,538,0.777690,0.651157,44.137145,67.782618,74.346792,77.918124,64.973284,87.819714,61198.814132,...,5.570141e-01,2.474841e+00,0.001206,1.139299e+08,0.0,1000000.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,625,0.843551,0.784001,13.959376,17.805311,17.807484,20.334814,20.109152,23.239152,2257.450217,...,5.000000e-01,1.000000e+00,0.000479,4.361832e+06,0.0,1000000.0,0.0,0.0,0.0,1
641,643,0.863104,0.593432,17.211454,29.003257,29.337226,26.632793,33.811070,36.594924,7328.440348,...,3.125000e-02,1.000000e+00,0.000166,3.606603e+07,0.0,1000000.0,0.0,0.0,0.0,1
642,543,0.436385,0.419528,24.462222,58.308908,30.586967,54.975914,55.109923,58.939915,22617.573935,...,8.199772e-10,-3.203427e-16,0.000029,0.000000e+00,0.0,1000000.0,0.0,0.0,0.0,1
643,351,0.782644,0.633827,14.731498,23.242127,23.899082,20.694753,26.035769,28.726272,3826.559124,...,3.466667e-01,1.584963e+00,0.000380,1.384783e+07,0.0,1000000.0,0.0,0.0,0.0,1


In [77]:
# validation set split

train_set, val_set = train_test_split(train_set, train_size = TRAIN_SPLIT, stratify = train_set['label'])

In [78]:
# transforms

transform = v2.Compose([
    tio.ZNormalization()
    # v2.Normalize(mean = [0.5], std = [0.225])
])

upscaler = tio.Resize(SIZE, 'bspline')

In [79]:
affine_args = {
        'scales' : [0.8, 1, 0.8, 1, 0.8, 1],
        'degrees' : 15,
        'translation' : [0.05, 0.05, 0.05],
        'center' : 'image'
}
augment = tio.Compose([
    tio.transforms.RandomAffine(
        **affine_args
    )
])

In [80]:
class_sample_count = train_set.label.value_counts()  
class_sample_count = 1/class_sample_count
class_sample_count = class_sample_count.tolist()

c_w = train_set.label.apply(lambda x: class_sample_count[x]).to_numpy()
c_w = torch.tensor(c_w, dtype = torch.double)

weighted_sampler = torch.utils.data.WeightedRandomSampler(c_w, len(c_w))

In [81]:
train_set.shape

(483, 109)

In [82]:
import torch
import pandas

import numpy
import os
import SimpleITK as sitk


def normalize_sitk_image(arr):
    '''Function to scale a simple-itk image array to values between 0 and 1
    
    Arguments:
    1. arr: a numpy array for a SimpleITK image
    
    Returns:
    1. scaled_arr: a scaled array with elements between 0 and 1.'''

    return (arr - arr.min())/(arr.max() - arr.min())

def convert(n):
    n = int(n)
    n+=1
    if(n>=0 and n<10):
        return "00"+str(int(n))
    
    elif(n>=10 and n<100):
        return "0"+str(int(n))
    
    return str(int(n))
    
    


class MRI_Dataset_within_ROI(torch.utils.data.Dataset):
    '''Dataset for loading MRI sequences with only the tumour ROI enclosed.'''
    def __init__(self,
                 src_path,
                 dataframe,
                 seg_bb,
                 transform,
                 upscale,
                 augment = None,
                 sequence = 'post_1.img.gz'
                 ):
        '''Init method

        Arguments:
        1. src_path: the path to the processed NIFTI Dataset
        2. dataframe: the file consisting of patient-class characterisations
        3. seg_bb: segmentation bounding boxes data (dataframe)
        4. transform: the transformation excluding the upscaling for the 3D volume
        5. upscale: the upscaling transform to convert sequences to a standard format
        6. augment: augmentation for the 3D voxel tensor
        7. sequence: the sequence name (eg. 'post_1.img.gz')

        Returns:
        1. MRI_Dataset_within_ROI dataset
        '''
        self.src_path = src_path
        self.df = dataframe.to_numpy()
        self.seg_bb = seg_bb
        self.transform = transform
        self.sequence = sequence
        self.upscale = upscale
        self.augment = augment

    def __len__(self):
        '''Function to get the length of the dataset'''
        return len(self.df)

    def __getitem__(self, idx):
        '''Function to fetch item at an index of the dataset

        Arguments:
        1. idx: index

        Returns:
        1. 3D tensor for the tumour volume
        2. label
        '''
        patient, label = self.df[idx][0],self.df[idx][-1]
        label = torch.tensor(label)
        
        path = os.path.join(self.src_path, "Breast_MRI_"+convert(patient), self.sequence)
        img = sitk.ReadImage(path)
        arr = sitk.GetArrayFromImage(img)
        row1, row2, col1, col2, slice1, slice2 = self.seg_bb.loc["Breast_MRI_"+convert(patient)].tolist()

        segment = torch.tensor(arr[slice1: slice2, row1: row2, col1: col2].astype('float32'))[None, ...]
        segment = self.transform(segment).type(torch.float32)
    
        segment = self.upscale(segment)
        if self.augment is not None:
            segment = self.augment(segment)

        label = label.type(torch.LongTensor)
        
        return segment, label
    
    
class MRI_Dataset_within_ROI_both_prepost(torch.utils.data.Dataset):
    '''Dataset for loading MRI sequences with only the tumour ROI enclosed.'''
    def __init__(self,
                 src_path,
                 dataframe,
                 seg_bb,
                 transform,
                 upscale,
                 augment = None
                 ):
        '''Init method

        Arguments:
        1. src_path: the path to the processed NIFTI Dataset
        2. dataframe: the file consisting of patient-class characterisations
        3. seg_bb: segmentation bounding boxes data (dataframe)
        4. transform: the transformation excluding the upscaling for the 3D volume
        5. upscale: the upscaling transform to convert sequences to a standard format
        6. augment: augmentation for the 3D voxel tensor

        Returns:
        1. MRI_Dataset_within_ROI dataset
        '''
        self.src_path = src_path
        self.df = dataframe.to_numpy()
        self.seg_bb = seg_bb
        self.transform = transform
        self.upscale = upscale
        self.augment = augment

    def __len__(self):
        '''Function to get the length of the dataset'''
        return len(self.df)

    def __getitem__(self, idx):
        '''Function to fetch item at an index of the dataset

        Arguments:
        1. idx: index

        Returns:
        1. 3D tensor for the tumour volume
        2. label
        '''
        patient, label = self.df[idx][0],self.df[idx][-1]
        label = torch.tensor(label)
        
        pre_path = os.path.join(self.src_path, "Breast_MRI_"+convert(patient), 'pre.img.gz')
        post_path = os.path.join(self.src_path, "Breast_MRI_"+convert(patient), 'post_1.img.gz')

    
        row1, row2, col1, col2, slice1, slice2 = self.seg_bb.loc["Breast_MRI_"+convert(patient)].tolist()
        pre_img = self.get_arrs(pre_path, row1, row2, col1, col2, slice1, slice2)
        post_img = self.get_arrs(post_path, row1, row2, col1, col2, slice1, slice2)

        
        segment = torch.concat([pre_img, post_img])
        
        if self.augment is not None:
            segment = self.augment(segment)

        label = label.type(torch.LongTensor)
        
        return segment, label
    

    def get_arrs(self, path, row1, row2, col1, col2, slice1, slice2):
        img = sitk.GetArrayFromImage(sitk.ReadImage(path))[slice1: slice2, row1: row2, col1: col2]
        img = torch.Tensor(img.astype('float32'))[None, ...]
        return self.upscale(self.transform(img))

In [83]:
train_dataset = MRI_Dataset_within_ROI_both_prepost(DATA_PATH,
                                       train_set,
                                       bounding_boxes, 
                                       transform,
                                       upscaler,
                                       augment = augment,
                                       )

val_dataset = MRI_Dataset_within_ROI_both_prepost(DATA_PATH,
                                       val_set,
                                       bounding_boxes, 
                                       transform,
                                       upscaler)

test_dataset = MRI_Dataset_within_ROI_both_prepost(DATA_PATH,
                                       test_set,
                                       bounding_boxes, 
                                       transform,
                                       upscaler)

In [84]:
train_set.shape

(483, 109)

In [85]:
bounding_boxes.iloc[0]

Start Row       234
End Row         271
Start Column    308
End Column      341
Start Slice      89
End Slice       112
Name: Breast_MRI_001, dtype: int64

In [86]:
train_loader_cnn = torch.utils.data.DataLoader(train_dataset,
                                        #    shuffle = True,
                                           batch_size = BATCH_SIZE,
                                           pin_memory = True,
                                           num_workers = NUM_WORKERS,
                                           sampler = weighted_sampler,
                                           persistent_workers = True
                                           )

val_loader_cnn = torch.utils.data.DataLoader(val_dataset,
                                           batch_size = BATCH_SIZE,
                                           pin_memory = True,
                                           num_workers = NUM_WORKERS,
                                           persistent_workers = True
                                           )

test_loader_cnn = torch.utils.data.DataLoader(test_dataset,
                                           batch_size = BATCH_SIZE,
                                           pin_memory = True,
                                           num_workers = NUM_WORKERS,
                                           persistent_workers = True
                                           )

In [87]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet_MRI3D(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ConvNet_MRI3D, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv2 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv3 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv4 = torch.nn.Conv3d(16, 32, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        
        # self.conv1x1_1 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        # self.conv1x1_2 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        self.maxpool1 = torch.nn.MaxPool3d(2)

        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(3456*2, num_classes)
    
    def forward(self, inp):
        out = inp
        out = F.relu(self.conv1(out))
        intermediate = self.maxpool1(out)
        
        
        out = F.relu(self.conv2(intermediate))
        out = out + intermediate      # residual
        intermediate = self.maxpool1(out)
        
        out = F.relu(self.conv3(intermediate))
        out = out + intermediate

        out = self.maxpool1(out)
        out = self.conv4(out)
        out = self.maxpool1(out)
        out = self.flatten(out)
        
        out = self.fc1(out)
        
        return out

In [88]:
weights = torch.tensor((train_set['label'].value_counts()/len(train_set)).to_numpy())
weights = 1 - weights
weights = weights.float()
print(weights)

tensor([0.2567, 0.7433])


In [89]:
torch.cuda.empty_cache()

In [91]:
from torch.utils.data import DataLoader, TensorDataset
train_set_tensor = torch.tensor(train_set.iloc[:, :-1].values, dtype=torch.float32)  # Excluding the last column (labels)
train_labels_tensor = torch.tensor(train_set.iloc[:, -1].values, dtype=torch.long)  # Extracting the labels

# Assuming val_set and test_set are similar DataFrames
val_set_tensor = torch.tensor(val_set.iloc[:, :-1].values, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_set.iloc[:, -1].values, dtype=torch.long)

test_set_tensor = torch.tensor(test_set.iloc[:, :-1].values, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_set.iloc[:, -1].values, dtype=torch.long)
batch_size = 25
# Create TensorDatasets
train_dataset = TensorDataset(train_set_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_set_tensor, val_labels_tensor)
test_dataset = TensorDataset(test_set_tensor, test_labels_tensor)
train_loader_lstm = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader_lstm = DataLoader(val_dataset, batch_size=batch_size)
test_loader_lstm = DataLoader(test_dataset, batch_size=batch_size)
# Training loop
num_epochs = 10

In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm import tqdm

# Define the Attention module
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        # Ensure both tensors have the same batch size
        batch_size = hidden.size(0)
        encoder_outputs = encoder_outputs[:, :batch_size, :]  # Adjust encoder_outputs if needed
        
        # Add a singleton dimension to encoder_outputs
        encoder_outputs = encoder_outputs.unsqueeze(2)  # Add a singleton dimension

        # Concatenate the hidden state with encoder outputs along the last dimension
        combined = torch.cat((hidden.unsqueeze(1), encoder_outputs), dim=-1)

        # Calculate the attention scores
        energy = torch.tanh(self.attn(combined))

        # Squeeze the attention scores to remove the added singleton dimension
        energy = energy.squeeze(2)

        # Calculate attention weights
        attention_weights = F.softmax(torch.matmul(energy, self.v), dim=1)

        # Apply attention weights to encoder outputs
        attended_encoder_outputs = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        return attention_weights, attended_encoder_outputs


# Define the combined model
class CombinedModel(nn.Module):
    def __init__(self, lstm_model, conv_model, hidden_size_lstm, num_classes=2):
        super(CombinedModel, self).__init__()
        self.lstm_model = lstm_model
        self.conv_model = conv_model
        self.fc1 = nn.Linear(hidden_size_lstm + num_classes, 256)  # Adjusted size for fc1
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x_lstm, x_conv):
        out_lstm = self.lstm_model(x_lstm)
        out_conv = self.conv_model(x_conv)

        # Adjust the size of out_lstm to match the second dimension of out_conv
        out_lstm = out_lstm[:, :out_conv.size(1)]  # Trim out_lstm if needed
        
        # Concatenate the LSTM output and ConvNet output
        combined_representation = torch.cat((out_lstm, out_conv), dim=1)
        # Pass the combined representation through the fully connected layers
        out = F.relu(self.fc1(combined_representation))
        out = self.fc2(out)
        return out


# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        return out

# Define the ConvNet model
class ConvNet_MRI3D(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ConvNet_MRI3D, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv2 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv3 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv4 = torch.nn.Conv3d(16, 32, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        
        # self.conv1x1_1 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        # self.conv1x1_2 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        self.maxpool1 = torch.nn.MaxPool3d(2)

        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(3456*2, num_classes)
    
    def forward(self, inp):
        out = inp
        out = F.relu(self.conv1(out))
        intermediate = self.maxpool1(out)
        
        
        out = F.relu(self.conv2(intermediate))
        out = out + intermediate      # residual
        intermediate = self.maxpool1(out)
        
        out = F.relu(self.conv3(intermediate))
        out = out + intermediate

        out = self.maxpool1(out)
        out = self.conv4(out)
        out = self.maxpool1(out)
        out = self.flatten(out)
        
        out = self.fc1(out)
        
        return out


# Instantiate the LSTM model
input_size_lstm = train_set_tensor.shape[1]  # Update this according to your input size
hidden_size_lstm = 64  # Change this according to your desired size
output_size_lstm = 2  # Change this according to your output size
lstm_model = LSTMModel(input_size_lstm, hidden_size_lstm, output_size_lstm)

# Instantiate the ConvNet model
in_channels_conv = 2  # Update this according to your input channels
num_classes_conv = 2  # Change this according to your output size
conv_model = ConvNet_MRI3D(in_channels_conv, num_classes_conv)

# Instantiate the combined model
combined_model = CombinedModel(lstm_model, conv_model,num_classes_conv)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(combined_model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    combined_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for (data_lstm, _), (data_conv, labels) in zip(train_loader_lstm, train_loader_cnn):
        optimizer.zero_grad()
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    epoch_loss = running_loss / len(train_loader_lstm.dataset)
    train_accuracy = correct_train / total_train

    # Validation
    combined_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for (data_lstm, _), (data_conv, labels) in zip(val_loader_lstm, val_loader_cnn):
            outputs = combined_model(data_lstm, data_conv)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted_val = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()
    val_loss /= len(val_loader_lstm.dataset)
    val_accuracy = correct_val / total_val

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

# Testing
test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for (data_lstm, _), (data_conv, labels) in zip(test_loader_lstm, test_loader_cnn):
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * labels.size(0)
        _, predicted_test = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted_test == labels).sum().item()
test_loss /= len(test_loader_lstm.dataset)
test_accuracy = correct_test / total_test

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")


/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 101, 100) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixi

Epoch 1/10, Train Loss: 0.4569, Train Acc: 0.8716, Val Loss: 0.5815, Val Acc: 0.7469
Epoch 2/10, Train Loss: 0.3543, Train Acc: 0.9130, Val Loss: 0.5799, Val Acc: 0.7469
Epoch 3/10, Train Loss: 0.3659, Train Acc: 0.8861, Val Loss: 0.5748, Val Acc: 0.7469
Epoch 4/10, Train Loss: 0.3644, Train Acc: 0.8861, Val Loss: 0.5802, Val Acc: 0.7469
Epoch 5/10, Train Loss: 0.3624, Train Acc: 0.8820, Val Loss: 0.5920, Val Acc: 0.7469
Epoch 6/10, Train Loss: 0.3765, Train Acc: 0.8737, Val Loss: 0.5743, Val Acc: 0.7469
Epoch 7/10, Train Loss: 0.4179, Train Acc: 0.8509, Val Loss: 0.5956, Val Acc: 0.7469
Epoch 8/10, Train Loss: 0.3351, Train Acc: 0.8965, Val Loss: 0.6404, Val Acc: 0.7407
Epoch 9/10, Train Loss: 0.3170, Train Acc: 0.8986, Val Loss: 0.6014, Val Acc: 0.7407
Epoch 10/10, Train Loss: 0.3104, Train Acc: 0.8986, Val Loss: 0.6366, Val Acc: 0.7099


/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 101, 100) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixi

Test Loss: 0.6323, Test Acc: 0.7329


In [96]:
torch.save(combined_model.state_dict(), MODEL_SAVE_PATH)

In [94]:
torch.cuda.empty_cache()

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm import tqdm

# Define the Attention module
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(4, hidden_size)
        self.v = nn.Parameter(torch.rand(25, 4))  # Adjusted to match the dimensions

    def forward(self, hidden, encoder_outputs):
        # Concatenate the hidden state with encoder outputs along the last dimension
        combined = torch.cat((hidden, encoder_outputs), dim=-1)
        # Calculate the attention scores
        energy = torch.tanh(self.attn(combined))
        # Calculate attention weights
        attention_weights = torch.matmul(energy, self.v)
        # attended_encoder_outputs = attended_encoder_outputs.transpose(0, 1)  # Transpose back
        return attention_weights




class CombinedModel(nn.Module):
    def __init__(self, lstm_model, conv_model, hidden_size_lstm, num_classes=2):
        super(CombinedModel, self).__init__()
        self.lstm_model = lstm_model
        self.conv_model = conv_model
        self.attention = Attention(hidden_size_lstm)  # Attention mechanism
        self.fc1 = nn.Linear(4, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x_lstm, x_conv):
        out_lstm = self.lstm_model(x_lstm)
        out_conv = self.conv_model(x_conv)

        # Adjust the size of out_lstm to match the second dimension of out_conv
        out_lstm = out_lstm[:, :out_conv.size(1)]  # Trim out_lstm if needed
        combined_representation = torch.cat((out_lstm, out_conv), dim=1)
        # Apply attention mechanism
        attended_combined_representation = self.attention(out_lstm, out_conv)
        combined_representation = torch.cat((out_lstm, out_conv), dim=1)
        # Pass the attended combined representation through the fully connected layers
        out = F.relu(self.fc1(attended_combined_representation))
        out = self.fc2(out)
        return out


# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        return out

# Define the ConvNet model
class ConvNet_MRI3D(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ConvNet_MRI3D, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv2 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv3 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv4 = torch.nn.Conv3d(16, 32, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        
        # self.conv1x1_1 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        # self.conv1x1_2 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        self.maxpool1 = torch.nn.MaxPool3d(2)

        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(3456*2, num_classes)
    
    def forward(self, inp):
        out = inp
        out = F.relu(self.conv1(out))
        intermediate = self.maxpool1(out)
        
        
        out = F.relu(self.conv2(intermediate))
        out = out + intermediate      # residual
        intermediate = self.maxpool1(out)
        
        out = F.relu(self.conv3(intermediate))
        out = out + intermediate

        out = self.maxpool1(out)
        out = self.conv4(out)
        out = self.maxpool1(out)
        out = self.flatten(out)
        
        out = self.fc1(out)
        
        return out


# Instantiate the LSTM model
input_size_lstm = train_set_tensor.shape[1]  # Update this according to your input size
hidden_size_lstm = 25  # Change this according to your desired size
output_size_lstm = 2  # Change this according to your output size
lstm_model = LSTMModel(input_size_lstm, hidden_size_lstm, output_size_lstm)

# Instantiate the ConvNet model
in_channels_conv = 2  # Update this according to your input channels
num_classes_conv = 2  # Change this according to your output size
conv_model = ConvNet_MRI3D(in_channels_conv, num_classes_conv)

# Instantiate the combined model
combined_model = CombinedModel(lstm_model, conv_model, hidden_size_lstm, num_classes_conv)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(combined_model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    combined_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for (data_lstm, _), (data_conv, labels) in zip(train_loader_lstm, train_loader_cnn):
        optimizer.zero_grad()
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    epoch_loss = running_loss / len(train_loader_lstm.dataset)
    train_accuracy = correct_train / total_train

    # Validation
    combined_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for (data_lstm, _), (data_conv, labels) in zip(val_loader_lstm, val_loader_cnn):
            outputs = combined_model(data_lstm, data_conv)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted_val = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()
    val_loss /= len(val_loader_lstm.dataset)
    val_accuracy = correct_val / total_val

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

# Testing
test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for (data_lstm, _), (data_conv, labels) in zip(test_loader_lstm, test_loader_cnn):
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * labels.size(0)
        _, predicted_test = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted_test == labels).sum().item()
test_loss /= len(test_loader_lstm.dataset)
test_accuracy = correct_test / total_test

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")


/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 101, 100) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 101, 100) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (101, 100, 100) != target shape (100, 100, 100). Fixi

Epoch 1/10, Train Loss: 0.5106, Train Acc: 0.7930, Val Loss: 0.6486, Val Acc: 0.6481
Epoch 2/10, Train Loss: 0.5714, Train Acc: 0.7495, Val Loss: 0.6710, Val Acc: 0.6481
Epoch 3/10, Train Loss: 0.5304, Train Acc: 0.7888, Val Loss: 0.6652, Val Acc: 0.6481
Epoch 4/10, Train Loss: 0.5392, Train Acc: 0.7826, Val Loss: 0.7065, Val Acc: 0.6481
Epoch 5/10, Train Loss: 0.4863, Train Acc: 0.8137, Val Loss: 0.7826, Val Acc: 0.6481
Epoch 6/10, Train Loss: 0.5347, Train Acc: 0.7805, Val Loss: 0.6986, Val Acc: 0.6481
Epoch 7/10, Train Loss: 0.5484, Train Acc: 0.7660, Val Loss: 0.6901, Val Acc: 0.6481
Epoch 8/10, Train Loss: 0.5817, Train Acc: 0.7433, Val Loss: 0.6711, Val Acc: 0.6481
Epoch 9/10, Train Loss: 0.5288, Train Acc: 0.7805, Val Loss: 0.7344, Val Acc: 0.6481
Epoch 10/10, Train Loss: 0.5574, Train Acc: 0.7702, Val Loss: 0.8679, Val Acc: 0.6481


/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 100, 101) != target shape (100, 100, 100). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/dhruv.kumar/anaconda3/envs/breast_cancer_py3.11/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Output shape (100, 101, 100) != target shape (100, 100, 100). Fixi

Test Loss: 0.8643, Test Acc: 0.6498


In [47]:
import torch.nn as nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, pool_proj):
        super(InceptionModule, self).__init__()
        
        # 1x1 conv branch
        self.branch1x1 = nn.Conv3d(in_channels, out_1x1, kernel_size=1)

        # 1x1 conv -> 3x3 conv branch
        self.branch3x3 = nn.Sequential(
            nn.Conv3d(in_channels, red_3x3, kernel_size=1),
            nn.Conv3d(red_3x3, out_3x3, kernel_size=3, padding=1)
        )

        # 1x1 conv -> 5x5 conv branch
        self.branch5x5 = nn.Sequential(
            nn.Conv3d(in_channels, red_5x5, kernel_size=1),
            nn.Conv3d(red_5x5, out_5x5, kernel_size=5, padding=2)
        )

        # 3x3 max pooling -> 1x1 conv branch
        self.branch_pool = nn.Sequential(
            nn.MaxPool3d(kernel_size=3, stride=1, padding=1),
            nn.Conv3d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(x)

        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        return torch.cat(outputs, 1)

class ConvNet_MRI3D(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ConvNet_MRI3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.inception1 = InceptionModule(16, 16, 16, 16, 16, 16, 16)
        self.inception2 = InceptionModule(64, 16, 16, 16, 16, 16, 16)
        self.inception3 = InceptionModule(64, 32, 32, 32, 32, 32, 32)
        
        self.maxpool1 = nn.MaxPool3d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(2048, num_classes)

    def forward(self, inp):
        out = inp
        out = F.relu(self.conv1(out))
        out = self.maxpool1(out)

        out = self.inception1(out)
        out = self.inception2(out)
        out = self.inception3(out)

        out = self.maxpool1(out)
        out = self.flatten(out)
        out = self.fc1(out)
        
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm import tqdm

# Define the Attention module
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        # Add a singleton dimension to encoder_outputs
        encoder_outputs = encoder_outputs.unsqueeze(2)  # Add a singleton dimension

        # Concatenate the hidden state with encoder outputs along the last dimension
        print(hidden.unsqueeze(1).shape)
        combined = torch.cat((hidden.unsqueeze(1), encoder_outputs), dim=-1)

        # Calculate the attention scores
        energy = torch.tanh(self.attn(combined))

        # Squeeze the attention scores to remove the added singleton dimension
        energy = energy.squeeze(2)

        # Calculate attention weights
        attention_weights = F.softmax(torch.matmul(energy, self.v), dim=1)

        # Apply attention weights to encoder outputs
        attended_encoder_outputs = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        return attention_weights, attended_encoder_outputs


class CombinedModel(nn.Module):
    def __init__(self, lstm_model, conv_model, hidden_size_lstm, num_classes=2):
        super(CombinedModel, self).__init__()
        self.lstm_model = lstm_model
        self.conv_model = conv_model
        self.attention = Attention(hidden_size_lstm)
        self.fc1 = nn.Linear(hidden_size_lstm + num_classes, 256)  # Adjusted size for fc1
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x_lstm, x_conv):
        out_lstm = self.lstm_model(x_lstm)
        out_conv = self.conv_model(x_conv)

        # Adjust the size of out_lstm to match the second dimension of out_conv
        out_lstm = out_lstm[:, :out_conv.size(1)]  # Trim out_lstm if needed
        
        # Concatenate the LSTM output and ConvNet output
        combined_representation = torch.cat((out_lstm, out_conv), dim=1)

        # Pass the combined representation through the attention mechanism
        attention_weights, attended_combined_representation = self.attention(combined_representation, combined_representation)

        # Pass the attended combined representation through the fully connected layers
        out = F.relu(self.fc1(attended_combined_representation))
        out = self.fc2(out)
        return out


# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        return out

# Define the ConvNet model
class ConvNet_MRI3D(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ConvNet_MRI3D, self).__init__()
        self.conv1 = torch.nn.Conv3d(in_channels, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv2 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv3 = torch.nn.Conv3d(16, 16, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        self.conv4 = torch.nn.Conv3d(16, 32, kernel_size = (3,3,3), stride = (1,1,1), padding = 1)
        
        # self.conv1x1_1 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        # self.conv1x1_2 = torch.nn.Conv3d(16, 16, kernel_size=1, stride=1)
        self.maxpool1 = torch.nn.MaxPool3d(2)

        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(3456*2, num_classes)
    
    def forward(self, inp):
        out = inp
        out = F.relu(self.conv1(out))
        intermediate = self.maxpool1(out)
        
        
        out = F.relu(self.conv2(intermediate))
        out = out + intermediate      # residual
        intermediate = self.maxpool1(out)
        
        out = F.relu(self.conv3(intermediate))
        out = out + intermediate

        out = self.maxpool1(out)
        out = self.conv4(out)
        out = self.maxpool1(out)
        out = self.flatten(out)
        
        out = self.fc1(out)
        
        return out


# Instantiate the LSTM model
input_size_lstm = train_set_tensor.shape[1]  # Update this according to your input size
hidden_size_lstm = 64  # Change this according to your desired size
output_size_lstm = 2  # Change this according to your output size
lstm_model = LSTMModel(input_size_lstm, hidden_size_lstm, output_size_lstm)

# Instantiate the ConvNet model
in_channels_conv = 2  # Update this according to your input channels
num_classes_conv = 2  # Change this according to your output size
conv_model = ConvNet_MRI3D(in_channels_conv, num_classes_conv)

# Instantiate the combined model
combined_model = CombinedModel(lstm_model, conv_model, hidden_size_lstm, num_classes_conv)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(combined_model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    combined_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for (data_lstm, _), (data_conv, labels) in zip(train_loader_lstm, train_loader_cnn):
        optimizer.zero_grad()
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    epoch_loss = running_loss / len(train_loader_lstm.dataset)
    train_accuracy = correct_train / total_train

    # Validation
    combined_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for (data_lstm, _), (data_conv, labels) in zip(val_loader_lstm, val_loader_cnn):
            outputs = combined_model(data_lstm, data_conv)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted_val = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()
    val_loss /= len(val_loader_lstm.dataset)
    val_accuracy = correct_val / total_val

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

# Testing
test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for (data_lstm, _), (data_conv, labels) in zip(test_loader_lstm, test_loader_cnn):
        outputs = combined_model(data_lstm, data_conv)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * labels.size(0)
        _, predicted_test = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted_test == labels).sum().item()
test_loss /= len(test_loader_lstm.dataset)
test_accuracy = correct_test / total_test

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")


torch.Size([25, 1, 4])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 1 but got size 4 for tensor number 1 in the list.